# ペットフードレビュー分類器 - クイックスタート

このノートブックでは、機械学習モジュールの基本的な使用方法を学びます。

## 🎯 このノートブックで学ぶこと

1. データの読み込みと前処理
2. 探索的データ分析（EDA）
3. BERTモデルの学習
4. モデルの評価と予測

## 📋 前提条件

- `ラベル付け参考用.xlsx` ファイルが準備されていること
- 必要なパッケージがインストールされていること (`pip install -r requirements.txt`)

## 1. 環境設定とインポート

In [ ]:
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# 現在のディレクトリをパスに追加
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.append(current_dir)

# 必要なモジュールのインポート
from data_preprocessing import DataPreprocessor
from eda import EDAAnalyzer
from bert_classifier import BERTClassifier
from config import MODEL_CONFIG, DATA_CONFIG

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch

print(f"PyTorch バージョン: {torch.__version__}")
print(f"CUDA利用可能: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")

print("\n✅ 環境設定完了！")

## 2. データファイルの確認

In [ ]:
# データファイルのパスを設定
excel_file = "../ラベル付け参考用.xlsx"  # パスを適切に調整してください

# ファイルの存在確認
if os.path.exists(excel_file):
    print(f"✅ データファイル確認: {excel_file}")
    
    # ファイル情報を表示
    file_size = os.path.getsize(excel_file) / 1024 / 1024  # MB
    print(f"   ファイルサイズ: {file_size:.2f} MB")
else:
    print(f"❌ ファイルが見つかりません: {excel_file}")
    print("   パスを確認してください")
    excel_file = None

## 3. データ前処理

In [ ]:
if excel_file and os.path.exists(excel_file):
    print("📊 データ前処理を開始...")
    
    # データ前処理器の初期化
    preprocessor = DataPreprocessor()
    
    # データの読み込みと前処理
    processed_data = preprocessor.process_data(excel_file)
    
    # 結果の表示
    print(f"\n📈 前処理結果:")
    print(f"   総データ数: {len(processed_data['texts'])}")
    print(f"   ラベル数: {len(processed_data['label_encoder'].classes_)}")
    print(f"   平均文字数: {np.mean([len(text) for text in processed_data['texts']]):.1f}")
    
    print(f"\n🏷️ ラベル一覧:")
    for i, label in enumerate(processed_data['label_encoder'].classes_):
        count = list(processed_data['original_labels']).count(label)
        percentage = count / len(processed_data['original_labels']) * 100
        print(f"   {i+1}. {label}: {count}件 ({percentage:.1f}%)")
        
    print("\n✅ データ前処理完了！")
else:
    print("❌ データファイルが利用できません")

## 4. 探索的データ分析（EDA）

データの分布や特徴を視覚的に確認します。

In [ ]:
if 'processed_data' in locals():
    print("📊 EDA分析を開始...")
    
    # EDA分析器の初期化
    eda_analyzer = EDAAnalyzer()
    
    # DataFrameを取得
    df = processed_data['dataframe']
    
    # 基本統計情報
    basic_stats = eda_analyzer.analyze_basic_statistics(df)
    
    print("\n✅ 基本統計情報分析完了！")
else:
    print("❌ 前処理されたデータがありません")

In [ ]:
# ラベル分布の可視化
if 'df' in locals():
    eda_analyzer.plot_label_distribution(df)
    print("📊 ラベル分布を表示中...")

In [ ]:
# テキスト長の分析
if 'df' in locals():
    eda_analyzer.plot_text_length_analysis(df)
    print("📏 テキスト長分析を表示中...")

In [ ]:
# クラス不均衡の分析
if 'df' in locals():
    imbalance_analysis = eda_analyzer.analyze_class_imbalance(df)
    print("⚖️ クラス不均衡分析完了！")

## 5. サンプルレビューの確認

各ラベルの実際のレビュー例を確認してみましょう。

In [ ]:
if 'df' in locals():
    eda_analyzer.show_sample_reviews(df, n_samples=3)
    print("\n👀 サンプルレビュー表示完了！")

## 6. BERT分類器の学習

いよいよBERTモデルを学習します。GPUがある場合は自動的に使用されます。

⚠️ **注意**: 初回実行時はBERTモデルのダウンロードに時間がかかります（数分〜10分程度）

In [ ]:
if 'processed_data' in locals():
    print("🤖 BERT分類器の学習を開始...")
    print("   ⏰ 初回実行時はモデルダウンロードに時間がかかります")
    
    # BERT分類器の初期化
    classifier = BERTClassifier()
    
    # 学習設定を表示
    print(f"\n⚙️ 学習設定:")
    print(f"   モデル: {MODEL_CONFIG['bert_model_name']}")
    print(f"   エポック数: {MODEL_CONFIG['num_epochs']}")
    print(f"   バッチサイズ: {MODEL_CONFIG['batch_size']}")
    print(f"   学習率: {MODEL_CONFIG['learning_rate']}")
    print(f"   使用デバイス: {'GPU' if torch.cuda.is_available() else 'CPU'}")
    
    # 学習実行（エポック数を少なめに設定）
    training_results = classifier.train(
        processed_data=processed_data,
        epochs=3,  # クイックスタートなので3エポック
        use_class_weights=True
    )
    
    print("\n🎉 学習完了！")
    print(f"   テスト精度: {training_results['test_results']['eval_accuracy']:.4f}")
    print(f"   テストF1: {training_results['test_results']['eval_f1']:.4f}")
    
else:
    print("❌ 前処理されたデータがありません")

## 7. モデルの評価と予測

学習したモデルで実際にテキストを分類してみましょう。

In [ ]:
if 'classifier' in locals():
    print("🧪 テストケースで予測を実行...")
    
    # テストケース
    test_cases = [
        {
            'text': '今までドライフードを色々試してきましたが、どれもちゃんと食べずに残してました。こちらは初めて与えた瞬間から飛び付いて喜んで完食してくれました。',
            'expected': '食べる'
        },
        {
            'text': 'うちの猫はこれを食べると、下痢でした',
            'expected': '吐く・便が悪くなる'
        },
        {
            'text': '配送の箱が破れていて中身がこぼれていました',
            'expected': '配送・梱包'
        },
        {
            'text': '値段が高くなってきて困ります',
            'expected': '値上がり/高い'
        },
        {
            'text': 'コスパが良くて助かります',
            'expected': '安い'
        }
    ]
    
    print(f"\n📝 テスト結果:")
    print("=" * 80)
    
    correct_count = 0
    
    for i, test_case in enumerate(test_cases, 1):
        # 予測実行
        prediction = classifier.predict(
            test_case['text'], 
            return_probabilities=True
        )
        
        # 結果の確認
        is_correct = prediction['predicted_label'] == test_case['expected']
        if is_correct:
            correct_count += 1
        
        # 結果表示
        print(f"\nテスト {i}:")
        print(f"   テキスト: {test_case['text'][:50]}...")
        print(f"   期待値: {test_case['expected']}")
        print(f"   予測: {prediction['predicted_label']} (確信度: {prediction['confidence']:.3f})")
        print(f"   結果: {'✅ 正解' if is_correct else '❌ 不正解'}")
    
    # 全体の正解率
    accuracy = correct_count / len(test_cases)
    print("\n" + "=" * 80)
    print(f"🎯 テストケース正解率: {accuracy:.1%} ({correct_count}/{len(test_cases)})")
    
else:
    print("❌ 学習済みモデルがありません")

## 8. 詳細な予測結果の確認

特定のテキストについて、全ラベルの確率を確認してみましょう。

In [ ]:
if 'classifier' in locals():
    # 複雑なケースをテスト
    complex_text = """前は他のフードを食べていましたが残すことが多くて困っていました。
    でもこちらに変えてからは完食してくれるようになり、うんちの調子も良くなりました。
    ただ、お値段が少し高いのが気になります。"""
    
    print("🔍 複雑なテキストの詳細分析:")
    print(f"テキスト: {complex_text}")
    print("\n" + "="*60)
    
    # 予測実行
    prediction = classifier.predict(complex_text, return_probabilities=True)
    
    # 最も確率の高いラベル
    print(f"\n🏆 最終予測: {prediction['predicted_label']} (確信度: {prediction['confidence']:.3f})")
    
    # 全ラベルの確率を表示
    print(f"\n📊 全ラベルの確率:")
    sorted_probs = sorted(
        prediction['all_probabilities'].items(), 
        key=lambda x: x[1], 
        reverse=True
    )
    
    for rank, (label, prob) in enumerate(sorted_probs, 1):
        bar = "█" * int(prob * 20)  # 確率を棒グラフで表現
        print(f"   {rank:2d}. {label:20s} {prob:.3f} {bar}")
        
else:
    print("❌ 学習済みモデルがありません")

## 9. インタラクティブな予測

自由にテキストを入力して予測してみましょう！

In [ ]:
if 'classifier' in locals():
    print("🎮 インタラクティブ予測モード")
    print("以下のセルを実行して、自由にテキストを入力してください！")
    print("（空文字を入力すると終了します）")
    
    def interactive_prediction():
        while True:
            user_text = input("\n予測したいレビューテキストを入力してください: ")
            
            if not user_text.strip():
                print("予測を終了します。")
                break
            
            try:
                prediction = classifier.predict(user_text, return_probabilities=True)
                
                print(f"\n📝 入力テキスト: {user_text}")
                print(f"🎯 予測ラベル: {prediction['predicted_label']}")
                print(f"📊 確信度: {prediction['confidence']:.3f}")
                
                # 上位3つの候補を表示
                sorted_probs = sorted(
                    prediction['all_probabilities'].items(), 
                    key=lambda x: x[1], 
                    reverse=True
                )[:3]
                
                print("\n🏆 上位候補:")
                for i, (label, prob) in enumerate(sorted_probs, 1):
                    print(f"   {i}. {label}: {prob:.3f}")
                    
            except Exception as e:
                print(f"❌ エラーが発生しました: {e}")
    
    # インタラクティブモードを開始
    # 注意: Jupyter環境では以下の行をコメントアウトしてください
    # interactive_prediction()
    
    print("\n💡 ヒント: 以下のようなテキストを試してみてください:")
    print("   - 'よく食べてくれます'")
    print("   - '全然食べません'")
    print("   - '届いた箱がへこんでいました'")
    print("   - 'もう少し安いといいのですが'")

else:
    print("❌ 学習済みモデルがありません")

## 10. まとめと次のステップ

### 🎉 完了したこと

1. ✅ データの読み込みと前処理
2. ✅ 探索的データ分析（EDA）
3. ✅ BERTモデルの学習
4. ✅ モデルの評価と予測
5. ✅ インタラクティブな予測テスト

### 📈 パフォーマンス向上のために

- **より多くのエポック**: `epochs=5` や `epochs=10` で学習
- **GPU使用**: より高速な学習
- **ハイパーパラメータ調整**: 学習率やバッチサイズの最適化
- **アンサンブル学習**: 複数モデルの組み合わせ

### 🚀 次のステップ

1. **本格的な学習**: `train.py`を使用した完全パイプライン
2. **モデルの保存と読み込み**: 学習済みモデルの再利用
3. **API化**: Webアプリケーションへの統合
4. **継続的改善**: 新しいデータでの再学習

### 💡 参考コマンド

```bash
# 完全パイプラインの実行
python train.py --excel_file "../ラベル付け参考用.xlsx" --epochs 10 --gpu

# 保存されたモデルの読み込み
classifier = BERTClassifier()
classifier.load_saved_model("./models/")
```

---

**🎊 お疲れ様でした！ペットフードレビュー分類器の基本的な使い方をマスターしました！**